<a href="https://colab.research.google.com/github/37stu37/FFE/blob/master/FFEpostProcessingRAPIDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Installation
---



In [10]:
!nvidia-smi # allocation must be Tesla T4, P4, or P100

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [11]:
# #no.of cores each processor is having 
# !lscpu | grep 'Core(s) per socket:'
# #no.of threads each core is having
# !lscpu | grep 'Thread(s) per core'

In [12]:
# %%time
# %%capture
# # Install RAPIDS
# !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
# !bash rapidsai-csp-utils/colab/rapids-colab.sh stable

# import sys, os

# dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
# sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
# sys.path
# exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

In [13]:
%%time
%%capture
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal
# Install rtree - Geopandas requirment
!apt install python3-rtree
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes
!pip install memory_profiler
# install parquet libraries
!pip install git+https://github.com/dask/fastparquet
!pip install python-snappy
!pip install pyarrow 

CPU times: user 252 ms, sys: 80.8 ms, total: 333 ms
Wall time: 57 s


In [14]:
from pathlib import Path
import os
from datetime import date

In [15]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import box
import dask.dataframe as dd
pd.options.mode.chained_assignment = None

In [16]:
# import cudf
# import dask_cudf

#Post-processing process 

1.   Load and concatenate parquet files into dask_cudf
2.   Count 'Source"
1.   Merge with Shapefile (gdf)
1.   Create output data









In [19]:
# Paths
pathShapefile = Path('/content/drive/My Drive/Colab Notebooks/01_Repository/FFE/data/shapefile')
pathParquets = Path('/content/drive/My Drive/Colab Notebooks/02_Output/ffeFinnComparison')

print(len(os.listdir(pathParquets)))

3000


In [20]:
def read_and_concatenate_parquets(path=pathParquets):
  L = []
  files = pathParquets.glob('*.parquet')
  number = 0
  for file in files:
    number += 1
    print(f"file loaded : {number}/{len(os.listdir(pathParquets))}")
    pqt = pd.read_parquet(file, engine='auto')
    pqt.drop_duplicates(subset ="source", inplace = True)
    L.append(pqt)
  df = dd.concat(L)
  return df

def count_fid_occurences(df):
  count = df['source'].value_counts().compute()
  count_df = pd.DataFrame({'source': count.index, 'count': count.values})
  count_df.to_parquet(str(pathShapefile) + '/' + f'CountBurn-{str(date.today())}.parquet', engine='auto', compression="GZIP")# could be datetime.now
  return count_df

def Merge(countDf,nameShapefile):
  # Shapefile
  gdfShape = gpd.read_file(pathShapefile / nameShapefile)
  gdfShape.insert(0, 'FID', range(0, len(gdfShape)))
  gdfShape.rename(columns={'FID': 'source'}, inplace=True)
  gdfShape = gdfShape[['source', 'geometry']]
  # gdf = pd.DataFrame(gdfShape)
  # cudf_gdf = cudf.DataFrame.from_pandas(gdf)
  # cudf_gdf.rename(columns={'FID': 'source'}, inplace=True)
  # cudf_gdf = cudf_gdf[['source', 'geometry']]
  # countDf as cudf
  # cudf_countDf = cudf.DataFrame.from_pandas(countDf)
  # merge
  merged = countDf.merge(gdfShape, on=['source'], how='left')
  return merged

def createShapefile(df):
  # df = df.to_pandas()
  gdf = gpd.GeoDataFrame(df, geometry='geometry')
  # gdf.plot(column='count', cmap='hsv', legend=True)
  gdf.to_file(os.path.join(str(pathShapefile) + "/" + "Burn3000scenarioWellington.shp"))
  return gdf

In [ ]:
%%time
concatDf = read_and_concatenate_parquets()

file loaded : 150/3000
file loaded : 151/3000
file loaded : 152/3000
file loaded : 153/3000
file loaded : 154/3000
file loaded : 155/3000
file loaded : 156/3000
file loaded : 157/3000
file loaded : 158/3000
file loaded : 159/3000
file loaded : 160/3000
file loaded : 161/3000
file loaded : 162/3000
file loaded : 163/3000
file loaded : 164/3000
file loaded : 165/3000
file loaded : 166/3000
file loaded : 167/3000
file loaded : 168/3000
file loaded : 169/3000
file loaded : 170/3000
file loaded : 171/3000
file loaded : 172/3000
file loaded : 173/3000
file loaded : 174/3000
file loaded : 175/3000
file loaded : 176/3000
file loaded : 177/3000
file loaded : 178/3000
file loaded : 179/3000
file loaded : 180/3000
file loaded : 181/3000
file loaded : 182/3000
file loaded : 183/3000
file loaded : 184/3000
file loaded : 185/3000
file loaded : 186/3000
file loaded : 187/3000
file loaded : 188/3000
file loaded : 189/3000
file loaded : 190/3000
file loaded : 191/3000
file loaded : 192/3000


In [ ]:
concatDf.head()

In [ ]:
%%time
countConcatDf = count_fid_occurences(concatDf)

In [ ]:
countConcatDf

In [ ]:
%%time
mergedDf = Merge(countConcatDf, 'WellWHV_Buildings.shp')
countShape = createShapefile(mergedDf)

In [ ]:
%%capture
!pip3 install contextily

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import contextily as ctx
%matplotlib inline
plt.style.use('seaborn-whitegrid')

FinnMeshblockShape = gpd.read_file(pathShapefile / 'Finn_MeshBlockSummary.shp')
countShape = gpd.read_file(pathShapefile / 'Burn3000scenarioWellington.shp')

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True, figsize=(12, 10))

p1 = originalShape.plot(ax=ax1, column='WellWHV_Bu', cmap='YlOrRd', alpha=0.7, legend=True)
ctx.add_basemap(ax1, crs=2193)
p2 = countShape.plot(ax=ax2, column='count', cmap='YlOrRd', alpha=0.7, legend=True)
ctx.add_basemap(ax2, crs=2193)

ax1.set_title('Original burn count at meshblock level')
ax1.ticklabel_format(useOffset=False, style='plain')
ax2.set_title('Network burn count at individual building level')
ax2.ticklabel_format(useOffset=False, style='plain')
ax1.tick_params(direction='out', length=6)
ax2.tick_params(direction="out", length=6)

fig.autofmt_xdate()
plt.tight_layout()
plt.savefig(pathShapefile / 'ComparisonGISvsNetwork_3000Burn.png', dpi=600)
plt.show()